# CxVISION 
This notebook will guide you on how to deploy the CxVision Model Package from AWS Marketplace to detect and track people in videos. 

## Usage instructions

Run this Jupyter notebook one cell at a time and press `Shift+Enter` to run each consecutive cell.

## Pre-requisites

1. This notebook renders correctly in Jupyter Notebook interface, so please open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that IAM role used has `AmazonSageMakerFullAccess` policy.
3. To deploy this ML model successfully, ensure that:
   1. You are able to make AWS Marketplace subscriptions and the IAM role has the following permissions:
      - `aws-marketplace:ViewSubscriptions`
      - `aws-marketplace:Unsubscribe`
      - `aws-marketplace:Subscribe`


## Contents

1. [Subscribe to the model package](#1.-subscribe-to-the-model-package)
2. [Create the model](#2.-Create-the-model)
3. [Deploy the model](#3.-Deploy-the-model)
4. [Perform inferences](#5.-Perform-inferences)
5. [Visualize results](#6.-Visualize-results)
6. [Clean Up your environment](#7.-Clean-Up-your-environment)


### 1. Subscribe to the model package

1. Open the model package page ["Customer experience vision"]().
2. Click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review it and click on **"Accept Offer"**.
4. Click on **Continue to configuration button** and then select a **region**. The **Product ARN** will be displayed. **This is the model package ARN that you need to specify while creating a deployable model using Boto3.**


### 2. Create the model

Create the model using the ARN of the CXVision Model Package

In [ ]:
from sagemaker import ModelPackage

sagemaker_role = '<YourSageMakerRole>' # Replace with your IAM Role for SageMaker
sagemaker_model_package_arn = '<ModelPackageARN>' # Replace this with your subcribed model package ARN

model = ModelPackage(role=sagemaker_role, 
                     model_package_arn=sagemaker_model_package_arn)

### 3. Deploy the model
The following cell deploys the created model on a real-time endpoint.

> Please, replace with your instance type. Supported instances types: ml.g4dn.xlarge, ml.g4dn.2xlarge, ml.g4dn.4xlarge, ml.g4dn.8xlarge, ml.g4dn.12xlarge

In [ ]:
import uuid

initial_instance_count=1    
instance_type='<InstanceType>' # Replace with your instance type. Supported instances types: ml.g4dn.xlarge, ml.g4dn.2xlarge, ml.g4dn.4xlarge, ml.g4dn.8xlarge, ml.g4dn.12xlarge
job_name='cxvision'

endpoint_name = '{}-{}'.format(job_name,str(uuid.uuid4()))
print('SageMaker Endpoint:',endpoint_name)
    
predictor = model.deploy(
    initial_instance_count=initial_instance_count,
    instance_type=instance_type,
    endpoint_name=endpoint_name
)

print('Enpoint name: {}'.format(endpoint_name))

### 4. Perform inferences
To invoke the endpoint, please ensure your video is in the folder `/videos` and follow the next steps.

#### 4.0 Preprocess your video
For better performance, your video must be prepared for the solution. 

This process will:
1. Rescale your video if the resolution is greater than 1280x72.
2. Generate a new video at 1 fps (Frames per second)

> Note: if the video to be preprocessed is large, the resources of the instance will likely need to be increased.

In [1]:
from utils.preprocess import *

video = "videos/example-video.mp4"
preprocess_video(video)

Video fps: 30
Original video Width: 1028 - Video Height: 732
0/45
1/45
2/45
3/45
4/45
5/45
6/45
7/45
8/45
9/45
10/45
11/45
12/45
13/45
14/45
15/45
16/45
17/45
18/45
19/45
20/45
21/45
22/45
23/45
24/45
25/45
26/45
27/45
28/45
29/45
30/45
31/45
32/45
33/45
34/45
35/45
36/45
37/45
38/45
39/45
40/45
41/45
42/45
43/45
44/45
45/45


#### 4.1 Real-time inferences

##### 4.1.1 Create the payload for real-time inferences

The content type of the required payload for starting to make inferences is `multipart/form-data`. This object has the following attributes:

* **video_name**: Indicates the name of the video to be processed. The required format is `<video-name>_area_<area-name>`. Example: `example_area_restaurant`. `area-name` can't have the underscore symbol. 
* **detection_threshold**: Indicates the threshold to consider a detection as a valid one. All the detections with a confidence equal to or higher than the detection_threshold will be taken. `It must be a float value between 5 and 100.`
* **blurring:** `True` if you want to apply blurring on the detected people. Otherwise, its value must be `False`.
* **timezone:**  Defines the timezone for processing the videos.
* **refresh_threshold:** Indicates the threshold for cleaning the endpoint variables. This value is expressed in hours. By default, its value is 1 (hour).
* **dwell_zone:** Defines the dwelling area of the video. Please, see how to define the areas in the [Define Areas Notebook](./utils/DefineAreas.ipynb).
* **service_zone:** Defines the service area of the video. Please, see how to define the areas in the [Define Areas Notebook](./utils/DefineAreas.ipynb).
* **video**: The video that will be processed by the solution. 

In [ ]:
# Update the object according to your data
import urllib3

video = "videos/preprocess_example-video.mp4"

payload, content_type = urllib3.encode_multipart_formdata({
    "video_name": "example_area_restaurant",
    "detection_threshold": "50",
    "blurring": "True",
    "timezone": "America/Los_Angeles",
    "refresh_threshold": "24",
    "dwell_zone": "[(200,500), (500,510), (490,710), (190,700)]",
    "service_zone": "[(540,580), (1000,645), (990,720), (530,655)]",
    "video": ("video.mp4", open(video, "rb").read(), "video/mp4")
})


##### 4.1.2 Invoke endpoint

The following code will invoke the endpoint to process the video specified in the payload. It would return a JSON object with the detection and tracking information and the generated metrics in form of logs.

- Real-time endpoint has a request timeout of 60 seconds.
- Input data must be at most 6MB.



In [ ]:
import boto3
import json

sm_runtime = boto3.client("sagemaker-runtime")

response_file = 'output/response.json'
response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload
)

with open(response_file, 'w') as outfile:
    outfile.write(json.dumps(json.load(response['Body'])))

> For an endpoint, limit the maximum size of the input data per invocation to 6 MB. 

#### 4.2 Perform batch inference

Upload the videos you want to process to an S3 bucket and start the batch transform job. For better performance, we recommend preprocessing each video before uploading it by executing the [Preprocess your video](#-4.0-preprocess-your-video)

In [ ]:
s3_data_path = "s3://cxvision/batch-process"
s3_output_path = 'YourS3OutputPath'
instance_type='InstanceType' # Replace with your instance type. Supported instances types: ml.g4dn.xlarge, ml.g4dn.2xlarge, ml.g4dn.4xlarge, ml.g4dn.8xlarge, ml.g4dn.12xlarge

transformer = model.transformer(
    instance_count=1,
    instance_type=instance_type,
    assemble_with="None",
    output_path=s3_output_path,
    max_concurrent_transforms=1,
    max_payload=6
)

transformer.transform(
    data=s3_data_path,
    data_type="S3Prefix",
    content_type="video/mp4"
)

> Currently, batch inference does not support zone capabilities. The detection and tracking will be done for all the people in the video.

### 5. Generate video with tracking results
The endpoint response contains the coordinates of the detections (tracking attribute) and the metrics in the form of logs (resume attribute). 

Example: [Model response](./sample/output/output-model-sample.json)

With the following cell, you will generate the final video by drawing bounding boxes in each video frame. 

In [9]:
from utils.show_results import *

output_video = "output/video-results-1.mp4"
generate_video(video, response_file, output_video)

Blurring people (if necessary)
{'y0': 269, 'y1': 659, 'x0': 326, 'x1': 439}
{'y0': 279, 'y1': 650, 'x0': 644, 'x1': 769}
{'y0': 357, 'y1': 690, 'x0': 946, 'x1': 1027}
{'y0': 418, 'y1': 720, 'x0': 364, 'x1': 478}
{'y0': 220, 'y1': 423, 'x0': 784, 'x1': 865}
{'y0': 231, 'y1': 353, 'x0': 776, 'x1': 892}
Drawing bounding boxes
Drawing tracking resume
Blurring people (if necessary)
{'y0': 266, 'y1': 644, 'x0': 349, 'x1': 463}
{'y0': 279, 'y1': 652, 'x0': 645, 'x1': 771}
{'y0': 357, 'y1': 691, 'x0': 946, 'x1': 1027}
{'y0': 412, 'y1': 715, 'x0': 388, 'x1': 501}
{'y0': 227, 'y1': 428, 'x0': 783, 'x1': 868}
{'y0': 231, 'y1': 353, 'x0': 775, 'x1': 892}
Drawing bounding boxes
Drawing tracking resume
Blurring people (if necessary)
{'y0': 264, 'y1': 642, 'x0': 357, 'x1': 471}
{'y0': 275, 'y1': 649, 'x0': 650, 'x1': 778}
{'y0': 343, 'y1': 691, 'x0': 946, 'x1': 1027}
{'y0': 392, 'y1': 700, 'x0': 411, 'x1': 526}
{'y0': 221, 'y1': 416, 'x0': 779, 'x1': 862}
{'y0': 227, 'y1': 356, 'x0': 773, 'x1': 894}


### 7. Clean Up your environment

#### 7.1. Delete model and endpoint

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.delete_model()

#### 7.2. Unsubscribe (optional)

1. Ensure that you do not have a [running model](https://console.aws.amazon.com/sagemaker/home#/models).
2. Go to __Machine Learning__ tab on [Your Software Subscriptions](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust) page.
2. Locate the listing that you want to cancel the subscription for and then choose __Cancel Subscription__.